In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns

# Algoritmalar
from sklearn.linear_model import LinearRegression

#Train ve Test Setlerinin ayrılması
from sklearn.model_selection import train_test_split

#Değerlendirme Metrikleri
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Gereksiz uyarıların görüntülenmemesi için kullanıyoruz
import warnings
warnings.filterwarnings('ignore')

data : https://www.kaggle.com/datasets/mirichoi0218/insurance

In [2]:
#İndirdiğimiz veri setini okuyoruz
df = pd.read_csv("./Data/insurance.csv")
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [4]:
# Sayısal değişkenlerin temel istatistiksel göstergeleri
df.describe().T

,count,mean,std,min,25%,50%,75%,max
age,1338.0,39.207025,14.049960,18.0000,27.00000,39.000,51.000000,64.00000
bmi,1338.0,30.663397,6.098187,15.9600,26.29625,30.400,34.693750,53.13000
children,1338.0,1.094918,1.205493,0.0000,0.00000,1.000,2.000000,5.00000
charges,1338.0,13270.422265,12110.011237,1121.8739,4740.28715,9382.033,16639.912515,63770.42801


In [5]:
#Tüm özellikler
#Hedef Özelliğimiz => charges (Veri setinde belirtildiğinden ötürü hedef özellik olduğunu biliyoruz)
df.columns

Index(['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges'], dtype='object')

In [6]:
df.region.unique()

array(['southwest', 'southeast', 'northwest', 'northeast'], dtype=object)

In [7]:
df.region.value_counts()

region
southeast    364
southwest    325
northwest    325
northeast    324
Name: count, dtype: int64

In [8]:
df.smoker.value_counts()

smoker
no     1064
yes     274
Name: count, dtype: int64

In [9]:
#Eksik gözlem kontrolü
df.isnull().mean()

age         0.0
sex         0.0
bmi         0.0
children    0.0
smoker      0.0
region      0.0
charges     0.0
dtype: float64

In [11]:
#Cinsiyete göre sağlık harcamalarının ortalaması
df.groupby(["sex"])["charges"].mean()

sex
female    12569.578844
male      13956.751178
Name: charges, dtype: float64

In [12]:
#Cinsiyete göre vücut kitle indeksi ortalaması
df.groupby(["sex"])["bmi"].mean()

sex
female    30.377749
male      30.943129
Name: bmi, dtype: float64

In [13]:
# One Hot Encoding yöntemiyle categorik değişkenlerimizi sayısal hale getiriyoruz. Çünkü modelimiz metinlerle çalışmıyor
one_hot_encoded_data = pd.get_dummies(df, columns = ['sex', 'smoker','region']) 

In [14]:
one_hot_encoded_data

,age,bmi,children,charges,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
0,19,27.900,0,16884.92400,True,False,False,True,False,False,False,True
1,18,33.770,1,1725.55230,False,True,True,False,False,False,True,False
2,28,33.000,3,4449.46200,False,True,True,False,False,False,True,False
3,33,22.705,0,21984.47061,False,True,True,False,False,True,False,False
4,32,28.880,0,3866.85520,False,True,True,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,False,True,True,False,False,True,False,False
1334,18,31.920,0,2205.98080,True,False,True,False,True,False,False,False
1335,18,36.850,0,1629.83350,True,False,True,False,False,False,True,False
1336,21,25.800,0,2007.94500,True,False,True,False,False,False,False,True


In [15]:
# hedef özelliğimizi geri kalan özelliklerden ayırıyoruz
# Bunu bağımlı değişkenimizi bağımsız değişkenlerden ayırıyoruz şeklinde de söyleyebiliriz
X = one_hot_encoded_data.drop(columns=["charges"])
y = one_hot_encoded_data["charges"]

In [16]:
# Verimizi train test olarak ikiye ayırıyoruz
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=42)

In [17]:
linear_reg = LinearRegression()
#Linear Regresyon modelini eğitiyoruz
linear_reg.fit(X_train,y_train)

LinearRegression()

In [21]:
#Sabit katsayı
linear_reg.intercept_

-574.5357125101764

In [22]:
# Diğer özelliklerin katsayıları
linear_reg.coef_

array([ 2.56975706e+02,  3.37092552e+02,  4.25278784e+02,  9.29584582e+00,
       -9.29584582e+00, -1.18255644e+04,  1.18255644e+04,  4.59585244e+02,
        8.89079180e+01, -1.98279052e+02, -3.50214110e+02])

In [18]:
# Eğittiğimiz Linear Regresyon modeli ile tahminde bulunuyoruz
y_pred = linear_reg.predict(X_test)

In [19]:
# Hata değerlerimiz
print("MAE:",mean_absolute_error(y_test,y_pred)) 
print("MSE:",mean_squared_error(y_test,y_pred))
print("RMSE:",np.sqrt(mean_squared_error(y_test,y_pred)))
print("R2 Score:",r2_score(y_test,y_pred))

MAE: 4181.194473753643
MSE: 33596915.85136147
RMSE: 5796.2846592762735
R2 Score: 0.7835929767120723


In [20]:
df_sonuc = pd.DataFrame(y_test)
df_sonuc["Tahmin"] = y_pred
# Hatayı mutlak değer içerisinde gösterdik
df_sonuc["Hata"] = abs(df_sonuc["charges"] - df_sonuc["Tahmin"])
df_sonuc

,charges,Tahmin,Hata
764,9095.06825,8969.550274,125.517976
887,5272.17580,7068.747443,1796.571643
890,29330.98315,36858.410912,7527.427762
1293,9301.89355,9454.678501,152.784951
259,33750.29180,26973.173457,6777.118343
...,...,...,...
109,47055.53210,39061.500932,7994.031168
575,12222.89830,11761.499198,461.399102
535,6067.12675,7687.563632,1620.436882
543,63770.42801,40920.291512,22850.136498
